In [ ]:
from model import *
from data import *

## Train your Unet with medical data - Pelvis
it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Set hyperparameters

* epochs_no - more runs and improvs generalisation of the model
* batch_size - number of samples to run at each epoch step

In [ ]:
# Hyper parametrs
batch_size = 8
epochs_no = 2000
steps_per_epoch = epochs_no / batch_size
epochs_no = 100

### Model Definition

In [ ]:
input_size = (256,256,1) # actual data
#input_size = (256,256,1) # Augmented data
model = unet(input_size=input_size)
model_checkpoint = ModelCheckpoint('unet_chest.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [ ]:
# callbacks = [ModelCheckpoint(filepathAcc, monitor='val_acc', ...),ModelCheckpoint(filepathLoss, monitor='val_loss', ...)]

### Train with data generator

In [ ]:
#if you don't want to do data augmentation, set data_gen_args as an empty dict.
#data_gen_args = dict()

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [ ]:
# Actual data size ~train, label folders and uses augmented saved data in ~aug folder
myGene = trainGenerator(batch_size,'data/chest/train','image','label',data_gen_args,save_to_dir = None)

# Downsampled data and no augmentation
#myGene = trainGenerator(batch_size,'data/chest/train','image_downsampled','label_downsampled',data_gen_args,save_to_dir = None)

In [ ]:
model.fit_generator(myGene,steps_per_epoch=steps_per_epoch,epochs=epochs_no,callbacks=[model_checkpoint])

### Train with npy file

### a. Train with Augmented data

In [ ]:
imgs_train,imgs_mask_train = geneTrainNpy("data/chest/train/aug/","data/chest/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=batch_sizes, epochs=epochs_no, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### b. Train with Original data

In [ ]:
imgs_train,imgs_mask_train = geneTrainNpy("data/chest/train/image_downsampled/","data/chest/train/label_downsampled/", image_prefix="",mask_prefix="")
model.fit(imgs_train, imgs_mask_train, batch_size=batch_size, epochs=epochs_no, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### Test your model and save predicted results

Link to Stack overflow discussion - https://stackoverflow.com/questions/52946110/u-net-low-contrast-test-images-predict-output-is-grey-box

### Set the output data dimension here

input and target size must match!

In [ ]:
## Resample to original sizes
input_size = (512,512,1)
target_size=(512,512)

### Test model on n datasets

In [ ]:
import os
test_path = 'data/chest/train/image/'
num_images = 0
for entry in os.listdir(test_path):
    if os.path.isfile(os.path.join(test_path, entry)):
        if entry.lower().endswith(('.png', '.jpg', '.jpeg')):
            num_images = num_images + 1

In [ ]:
# Path toread datasets tp apply prediction to
testGene = testGenerator(test_path,num_image=num_images, target_size=target_size)

In [ ]:
model = unet(input_size=input_size)
model.load_weights("unet_chest.hdf5")
results = model.predict_generator(testGene,num_images,verbose=1)

### Save the predicted data

In [ ]:
# Input is image and path to save the predicted datasets to
saveResult("data/chest/test_orig_label",results)